# Data Analysis Extra - Playlists

Tras el Análisis de features donde no tuve los resultados esperados, decidí hacer un análisis con las playslist que crearon los usuarios durante la pandemia y compararlo con playlists del 2019.

El proceso es: buscar las playlists que tengan en el nombre alguna palabra relacionada con la pandemia (me he limitado al español, pero ha sido inevitable coger a nieval global) y extraer todas sus canciones para luego hacer un proceso de análisis de Featuress.

IMPORTANTE: Hay celdas que ejecutan extracciones que tardan hasta una hora y media en completarse, por lo que no hace falta ejecutarlas, y lo resultante lo he exportado en CSVs en la carpeta raiz para luego poder cargarlos.

## Preparación Previa

In [393]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
from datetime import datetime
import time
import altair as alt

# Meter credenciales de Spotify
passw = pd.read_csv("pass_spotify.txt", sep = ',', encoding="utf-8")
client_id = passw.columns[0]
client_secret = passw.columns[1]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Cargamos funciones de anteriores Notebooks simplificadas para extraer features

In [394]:
##### Función para sacar las features de canciones que me interesan desde la id
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  valence = features[0]['valence']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  id = features[0]['id']

  track = [name, album, artist, release_date, length, popularity,
           acousticness, danceability, energy, instrumentalness,
           liveness, loudness, speechiness, valence, tempo, time_signature, id]
  return track


##### Loop para sacar features de todas las canciones de una lista, devuelve un dataframe

def extract_songs(list_toextract):
    import time
    tracks = []
    for i in range(len(list_toextract)):
        time.sleep(.5)
        track = getTrackFeatures(list_toextract[i])
        tracks.append(track)

    # Metemos la info en Dataframe
    data_features = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date',
                                                 'length', 'popularity','acousticness', 'danceability', 'energy',
                                                 'instrumentalness', 'liveness', 'loudness',
                                                 'speechiness', 'valence','tempo', 'time_signature', 'id'])

    data_features_final = data_features [['name','artist','album','release_date','length', 'popularity',
                                                 'acousticness', 'danceability', 'energy',
                                                 'instrumentalness', 'liveness', 'loudness',
                                                 'speechiness', 'valence','tempo', 'time_signature', 'id']]

    data_features_final = data_features_final.rename(columns = {'id':'spotify_id'})

    # Finalmente meto el año de Release date (lanzamiento de la canción) en un nueva columna que me va a vernir en uno de los análisis
    data_features_final['release_date_year'] = pd.to_datetime(data_features_final['release_date'])
    data_features_final['release_date_year'] = pd.DatetimeIndex(data_features_final['release_date']).year
    print(data_features_final.shape)
    data_features_final.head(4)

    #Normalizamos features
    features_to_normalize = ['length', 'popularity', 'loudness', 'tempo', 'speechiness']

    data_features_final[features_to_normalize] = data_features_final[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

    return data_features_final
    



# Pregunta 4: ¿Qué tipo de música contienen las playlists que se han creado para el tiempo de pandemia?, ¿han cambiado?

Lo primero que vamos a hacer es buscar playlists que se hayan creado "para" el coronavirus. Para eso, nos vamos a fijar en sus títulos: si contienen alguna palabra de las siguientes, nos las quedamos: "covid", "pandemia", "cuarentena",'covid19', "coronavirus", 'fernando simón'. No he podido meter a Fernando Simón en el ajo.

Posteriormente haremos el proceso de extracción de todo: playlists, canciones y features de canciones. Del total de canciones(hay muchas) de las playlist del coronavirus nos quedaremos solo con las más repetidas y en dos lotes, uno pequeño(top100) y otro más grande(top10000). La idea es comparar estos dos lotes con el Top100 del 2019, que es una playlist con las canciones más escuchadas durante del 2019 que ha creado Spotify y que le agradecemos enormemente.

In [395]:
# Vemos como está montada la estructura de información en la API de Spotify, para buscar una palabra en los nombres de las playlists
print(sp.search('q=fernando simón', type='playlist'))
# ["covid","cuarentena", 'pandemia','covid19', "coronavirus", 'fernando simón']

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=q%3Dfernando+sim%C3%B3n&type=playlist&offset=0&limit=10', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.co

Creamos un script ya completo para buscar las palabras y sacar solo el total de listas que contiene esta palabra en alguna parte de su información

In [396]:
for word in ["fernando simón"]:
    print(sp.search('q="{}"'.format(word), type='playlist')['playlists']['items'])

[{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1/tracks', 'total': 271}, 'type': 'playlist', 'uri': 'spotify:playlist:01bawF2DM0YzefGbC54Ua1'}

## Extraer playlists del coronavirus

Hacemos una extracción masiva de dichas playlist con los terminos "covid","pandemia","cuarentena",'covid19', "coronavirus", 'fernando simón'

In [397]:
# IMRPOTANTE!! Tiempo de ejecición: 1 minuto
# Documentación: https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlist/
# ['items']["tracks"]['items'][0]["added_at"]

# Función para extrar determinados features de la playlist
def batch_proccess(x,lista):
    for i in x['playlists']['items']:
        lista.append({"name" : i['name'], "total":i["tracks"]['total'], 'id':i['id'], "uri":i["uri"], "URL":i["tracks"]['href']})

# Sacar un listado de las playlist, en bloques de 50, que es el máximo que te permite la API
Time1 = datetime.now()

list_of_playlists = []
for term in ["covid","pandemia","cuarentena",'covid19', "coronavirus", 'fernando simón']:
    for i in range(0,2000, 50):
        try:
            init_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
            batch_proccess(init_data,list_of_playlists)
        except:
            print("Error")
Time2 = datetime.now()

print("Tiempo ejecución:", Time2 -Time1)

# Meto las playlists en un dataframe y echo un ojo
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)
print('Número de playlists:', playlists.count())
playlists.head(20)

Tiempo ejecución: 0:00:44.717344
Número de playlists: name     5557
total    5557
id       5557
uri      5557
URL      5557
dtype: int64


,name,total,id,uri,URL
0,quarantine Vibes | Covid 19 |,45,6zjC569rXoYGOLwPUt37WW,spotify:playlist:6zjC569rXoYGOLwPUt37WW,https://api.spotify.com/v1/playlists/6zjC569rX...
1,Calma,151,37i9dQZF1DWY5LGZYBBHHz,spotify:playlist:37i9dQZF1DWY5LGZYBBHHz,https://api.spotify.com/v1/playlists/37i9dQZF1...
2,COVID-19 Quarantine Party,29,1Tn6OrkJhOYO5u6y16erqB,spotify:playlist:1Tn6OrkJhOYO5u6y16erqB,https://api.spotify.com/v1/playlists/1Tn6OrkJh...
3,Canciones Mas Virales que el Coronavirus 🦠🎶,106,4Y2h9TZNZersxSYeXujCNu,spotify:playlist:4Y2h9TZNZersxSYeXujCNu,https://api.spotify.com/v1/playlists/4Y2h9TZNZ...
4,Drunksouls COVID19 quarantine discovery: music...,49,64KUBwKGkUEqIU08HDhJ94,spotify:playlist:64KUBwKGkUEqIU08HDhJ94,https://api.spotify.com/v1/playlists/64KUBwKGk...
5,Plenas que no pudimos bailar por COVID,81,2U4J1vAVC8DvV3fPfrgXVy,spotify:playlist:2U4J1vAVC8DvV3fPfrgXVy,https://api.spotify.com/v1/playlists/2U4J1vAVC...
6,chill beats to quarantine to,124,5bPe4ZISvKWU5yTzsSO9lX,spotify:playlist:5bPe4ZISvKWU5yTzsSO9lX,https://api.spotify.com/v1/playlists/5bPe4ZISv...
7,Quarantine covid - 19,100,7qJzyN6atsM49NOWnqGbEK,spotify:playlist:7qJzyN6atsM49NOWnqGbEK,https://api.spotify.com/v1/playlists/7qJzyN6at...
8,MÁS PERREO MENOS COVID,99,1RYr3ngXNoPVtl7blr9Kaf,spotify:playlist:1RYr3ngXNoPVtl7blr9Kaf,https://api.spotify.com/v1/playlists/1RYr3ngXN...
9,CORONAVIRUS QUARANTINE PARTY COVID-19 2020,55,3r9u5bXzAYEvc0xBKkSnAx,spotify:playlist:3r9u5bXzAYEvc0xBKkSnAx,https://api.spotify.com/v1/playlists/3r9u5bXzA...


## Extraer canciones de una cantidad de playlists del coronavirus

Hay más de 5000 playlists por lo que sólo vamos a recoger un rango

In [398]:
# Número de playlists de las que extraer canciones
playlists_extract = playlists[0:4000]

Hacemos la extracción de canciones de las playlists

In [399]:
# IMPORTANTE, Tiempo de ejecución: Alrededor de 30min

# Extracción canciones
Time1 = datetime.now()
songs = []
for uri in playlists_extract['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    songs.append(k['track']["id"])
            except:
                pass

Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)

print(len(songs))

Tiempo ejecución: 0:25:49.776226
467881


Hacemos un ranking de canciones más repetidas

In [465]:
# Vamos a hacer un de las canciones más repetidas
from collections import Counter
data_playlist_songs = pd.DataFrame.from_dict(Counter(songs), orient='index').reset_index().rename(columns={"index":"spotify_id", 0:"count"})
data_playlist_songs = data_playlist_songs.sort_values(by=['count'], ascending=False)
data_playlist_songs

,spotify_id,count
251,2DEZmgHKAvm41k4J3R2E9Y,466
326,0SqqAgdovOE24BzxIClpjw,451
6184,None,373
400,7k4t7uLgtOxPwTpFmtJNTY,363
1634,6NfrH0ANGmgBXyxgV2PeXt,294
...,...,...
92374,5fgHGI9EBaQkggzi6Zaq30,1
92375,00Jrzl6fENQd3XEWGDtowy,1
92376,2sKj6wgBUnAQmRwDZkE6ND,1
92377,5MkZkFLnUrsMrVKQwqv5zV,1


## Top15 pandemia playlists y Top15 oldies pandemia playlist

Por curiosidad, podemos echar un vistazo a cuáles son las top15 de la cuarentena. Hay que tener en cuenta de que el listado de playlist era de caracter mundial.

In [474]:
#Seleccionamos los 15 primeros
list_toextract_exitos = data_playlist_songs["spotify_id"].tolist()
list_toextract_exitos = list_toextract_exitos[0:11]
print(len(list_toextract_exitos))

# Quitar Nones, hay uno suelto
list_toextract_exitos = list(filter(None, list_toextract_exitos)) 
print(len(list_toextract_exitos))

# Extraer canciones usando la función creada inicialmente
top_songs_pandemia = extract_songs(list_toextract_exitos)
top_songs_pandemia

11
10
(10, 18)


,name,artist,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,spotify_id,release_date_year
0,Safaera,Bad Bunny,YHLQMDLG,2020-02-28,1.000000,0.266667,0.01030,0.607,0.829,0.000000,0.1070,0.745668,1.000000,0.685,0.025090,4,2DEZmgHKAvm41k4J3R2E9Y,2020
1,Yo Perreo Sola,Bad Bunny,YHLQMDLG,2020-02-28,0.104727,0.266667,0.02100,0.860,0.758,0.000065,0.3440,0.573041,0.049447,0.453,0.034897,4,0SqqAgdovOE24BzxIClpjw,2020
2,Tusa,KAROL G,Tusa,2019-11-07,0.314509,0.400000,0.29500,0.803,0.715,0.000134,0.0574,0.871880,0.751229,0.574,0.082316,4,7k4t7uLgtOxPwTpFmtJNTY,2019
3,La Difícil,Bad Bunny,YHLQMDLG,2020-02-28,0.038933,0.066667,0.08610,0.685,0.848,0.000007,0.0783,0.668256,0.099509,0.761,1.000000,4,6NfrH0ANGmgBXyxgV2PeXt,2020
4,Amarillo,J Balvin,Colores,2020-03-19,0.000000,0.000000,0.01300,0.641,0.857,0.005340,0.0695,0.483230,0.760442,0.961,0.334413,5,6zEgnpM0qYmHLDnh8WPejL,2020
5,Blinding Lights,The Weeknd,After Hours,2020-03-20,0.307815,1.000000,0.00146,0.514,0.730,0.000095,0.0897,0.450008,0.019656,0.334,0.896741,4,0VjIjW4GlUZAMYd2vXMi3b,2020
6,Azul,J Balvin,Colores,2020-03-19,0.350691,0.333333,0.08160,0.843,0.836,0.001380,0.0532,1.000000,0.049447,0.650,0.000000,4,2lCkncy6bIB0LTMT7kvrD1,2020
7,Say So,Doja Cat,Hot Pink,2019-11-07,0.583224,0.333333,0.25600,0.787,0.673,0.000004,0.0904,0.665713,0.321253,0.786,0.197363,4,3Dv1eDb0MEgF93GpLXlucZ,2019
8,death bed (coffee for your head),Powfu,death bed (coffee for your head),2020-02-08,0.113502,0.666667,0.73100,0.726,0.431,0.000000,0.6960,0.000000,0.250614,0.348,0.582491,4,7eJMfftS33KTjuF7lTsMCx,2020
9,Supalonely,BENEE,STELLA & STEVE,2019-11-15,0.478358,0.333333,0.30500,0.863,0.631,0.000030,0.1230,0.647910,0.000000,0.817,0.407201,4,4nK5YrxbMGZstTLbvj6Gxw,2019


Si cogemos un rango más grande y buscamos canciones antiguas (anteriores al 2000) para sacar un top10 de clásicos, nos encontramos con temazos curiosos y muy relacionados con la "experiencia coronavirus". Me parecen muy lógicas las canciones "It's The End Of The World As We Know It", "Stayin' Alive" y "Highway to Hell"

In [477]:
#Seleccionamos 100 primeros
list_toextract_exitos_oldies = data_playlist_songs["spotify_id"].tolist()
list_toextract_exitos_oldies = list_toextract_exitos_oldies[0:200]
print(len(list_toextract_exitos_oldies))

# Quitar Nones, hay uno suelto
list_toextract_exitos_oldies = list(filter(None, list_toextract_exitos_oldies)) 
print(len(list_toextract_exitos_oldies))

# Extraer canciones usando la función creada inicialmente
top_songs_pandemia_oldies = extract_songs(list_toextract_exitos_oldies)
top_songs_pandemia_oldies = top_songs_pandemia_oldies[top_songs_pandemia_oldies['release_date_year'] < 2000]
top_songs_pandemia_oldies

200
199
(199, 18)


,name,artist,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,spotify_id,release_date_year
46,Don't Stand So Close To Me,The Police,Zenyatta Mondatta (Remastered 2003),1980-10-03,0.460847,0.438596,0.04530,0.799,0.506,0.001070,0.0365,0.563892,0.079483,0.518,0.505883,4,5veJDT0MLsLbhYsx42GXUD,1980
48,It's The End Of The World As We Know It (And I...,R.E.M.,Document (R.E.M. No. 5),1987-09-01,0.473974,0.456140,0.02580,0.381,0.894,0.000000,0.0251,0.606052,0.027808,0.797,1.000000,4,2oSpQ7QtIKTNFfA08Cy0ku,1987
50,U Can't Touch This,MC Hammer,Please Hammer Don't Hurt 'Em,1990-02-20,0.506051,0.526316,0.00456,0.867,0.517,0.000339,0.0864,0.357003,0.138384,0.866,0.451629,4,1B75hgRqe7A4fwee3g3Wmu,1990
85,Stayin' Alive,Various Artists,Staying Alive (Original Motion Picture Soundtr...,1977-12-13,0.000000,0.368421,0.11300,0.707,0.535,0.006150,0.0884,0.000000,0.042260,0.641,0.228099,4,4UDmDIqJIbrW0hMBQMFOsM,1977
109,Every Breath You Take,The Police,Synchronicity (Remastered 2003),1983-06-17,0.495468,0.701754,0.54300,0.820,0.452,0.002940,0.0714,0.543138,0.022991,0.740,0.332326,4,1JSTJqkT5qHq8MDJnJbRE1,1983
119,Take on Me,a-ha,Hunting High and Low,1985-06-01,0.407360,0.701754,0.01800,0.573,0.902,0.001250,0.0928,0.660382,0.065032,0.876,0.082392,4,2WfaOiMkCvy7F5fcp2zZ8L,1985
155,Another One Bites The Dust - Remastered 2011,Queen,The Game (2011 Remaster),1980-06-27,0.374667,0.666667,0.11200,0.933,0.528,0.312000,0.1630,0.723731,0.299321,0.754,0.276004,4,5vdp5UmvTsnMEMESIF2Ym7,1980
162,Africa,TOTO,Toto IV,1982-04-08,0.624595,0.701754,0.25700,0.671,0.373,0.000079,0.0481,0.093937,0.017517,0.732,0.145313,4,2374M0fQpWi3dLnB54qaLX,1982
176,Should I Stay or Should I Go - Remastered,The Clash,Combat Rock (Remastered),1982,0.295705,0.631579,0.07900,0.742,0.833,0.000000,0.3840,0.724220,0.198599,0.816,0.301824,4,39shmbIHICJ2Wxnk1fPSdz,1982
197,Highway to Hell,AC/DC,Highway to Hell,1979-07-27,0.355430,0.701754,0.06100,0.574,0.913,0.001580,0.1560,0.814952,0.238012,0.423,0.319651,4,2zYzyRzz6pRmhPzyfMEC8s,1979


## Extracción Features de canciones de Top100 y Top10000 de las playlists de la pandemia

Volvemos a neustro análisis...Hacemos paquetes con los top100 y top10000 porque son extraccione slargas de información

In [478]:
# Cogemos las 100 canciones que salen más veces y luego las 10000 que salen más veces para extraer sus features
data_playlist_songs_100 = data_playlist_songs[0:100]
data_playlist_songs_100 = data_playlist_songs_100["spotify_id"].tolist()
data_playlist_songs_100 = list(filter(None, data_playlist_songs_100)) 
print(len(data_playlist_songs_100))

data_playlist_songs_10000 = data_playlist_songs[0:10000]
data_playlist_songs_10000 = data_playlist_songs_10000["spotify_id"].tolist()
data_playlist_songs_10000 = list(filter(None, data_playlist_songs_10000)) 
print(len(data_playlist_songs_10000))

99
9999


Sacamos las features de cada canción y metemos la información en un dataframe

In [404]:
# IMPORTANTE!!! Tiempo de ejecución: 1 minuto
# NO HACE FALTA EJECUTARLO, hay un CSV exportado con el resultado y que luego cargamos: data_ana_playlist_songs100.csv

# Ejecutamos funciones de extraer canciones y normalizar de PAQUETE 1: Top100
Time1 = datetime.now()
data_features_songs100 = extract_songs(data_playlist_songs_100)
Time2 = datetime.now()

# Exportamos
data_features_songs100.to_csv("data_ana_playlist_songs100.csv", sep = ',')

print("Tiempo ejecución extracción:", Time2 -Time1)
data_features_songs100.shape

(99, 18)
Tiempo ejecución extracción: 0:00:57.005631


(99, 18)

Echamos un vistazo a las medias, que en ste caso no podemos ponderar porque no tenemos los Streams totales

In [405]:
# IMPORTANTE!!! Tiempo de ejecución: 1 hora y media
# NO HACE FALTA EJECUTARLO, hay un CSV exportado con el resultado y que luego cargamos: data_ana_playlist_songs10000.csv

#Ejecutamos funciones de extraer canciones y normalizar de PAQUETE 2: 10000

Time1= datetime.now()

data_features_songs10000 = extract_songs(data_playlist_songs_10000)

Time2= datetime.now()

# Ha tardado casi 1 hora en extraerse así que lo guardamos CSV
data_features_songs10000.to_csv("data_ana_playlist_songs10000.csv", sep = ',')

print("Tiempo ejecución extracción 100:", Time2 -Time1)

(9999, 18)
Tiempo ejecución extracción 100: 1:35:57.610616


In [480]:
data_features_songs10000.head(3)

,name,artist,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,spotify_id,release_date_year
0,Safaera,Bad Bunny,YHLQMDLG,2020-02-28,0.193178,0.87,0.0103,0.607,0.829,0.000000,0.1070,0.805253,0.404951,0.685,0.283458,4,2DEZmgHKAvm41k4J3R2E9Y,2020
1,Yo Perreo Sola,Bad Bunny,YHLQMDLG,2020-02-28,0.104320,0.87,0.0210,0.860,0.758,0.000065,0.3440,0.767371,0.053486,0.453,0.288479,4,0SqqAgdovOE24BzxIClpjw,2020
2,Tusa,KAROL G,Tusa,2019-11-07,0.125141,0.90,0.2950,0.803,0.715,0.000134,0.0574,0.832950,0.312968,0.574,0.312755,4,7k4t7uLgtOxPwTpFmtJNTY,2019


## Extracción de canciones y features de playlist Top2019

Sacamos canciones y features del TopTrack 2019 de Spotify, que es una lista concreta de Spotify: https://open.spotify.com/playlist/37i9dQZF1DWTqOoGdpVCf0?si=YuItUkwFTyijpmbN4UuZVg

In [406]:
# Toptrack2019
songs_top19 = []
uri = 'spotify:playlist:37i9dQZF1DWTqOoGdpVCf0'
lenght = sp.playlist_tracks(uri)['total']
for i in range(0,lenght, 50):
    init_data = sp.playlist_tracks(uri, limit=50, offset=i)
    try:
        for k in init_data['items']:
            songs_top19.append(k['track']["id"])
    except:
        pass

#Ejecuto función    
Time1 = datetime.now()

data_features_songs100_2019 = extract_songs(songs_top19)

Time2 = datetime.now()

# Ha tardado casi 1 hora en extraerse así que lo guardamos CSV
data_features_songs100_2019.to_csv("data_ana_playlist_songs100_2019.csv", sep = ',')

print("Tiempo ejecución extracción top2019:", Time2 -Time1)
print(data_features_songs100_2019.shape)
data_features_songs100_2019.head(2)

(100, 18)
Tiempo ejecución extracción top2019: 0:00:57.643427
(100, 18)


,name,artist,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,spotify_id,release_date_year
0,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,0.280950,1.000000,0.751,0.501,0.405,0.00,0.105,0.749490,0.006459,0.446,0.306367,4,7qEHsqek33rTcFNT9PFqLf,2019
1,bad guy,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",2019-03-29,0.330765,0.977528,0.328,0.701,0.425,0.13,0.100,0.300611,0.770601,0.562,0.505711,4,2Fxmhks0bxGSBdJ92vM42m,2019


## Cálculo de medias de mi Análisis propio en Notebook Data_Analysis1_Features

Aprovechamos y para tener una información comparativa extra traemos las medias ponderadas de las features de las canciones analizadas en el primer Notebook, que son los Top200 diarios en España.


In [462]:
data_ana_corona = pd.read_csv("data_global_coronaperiod.csv", sep = ',')

# Quitamos algunas columnas que no usaremos y sobran
data_ana_corona = data_ana_corona.drop(columns=['Unnamed: 0', 'Position'])
print('Tamaño quitando columnas que sobran: ', data_ana_corona.shape)

# Quitamos los duplicados por canciones, no nos sirven de momento para los análisis
data_ana_corona = data_ana_corona.drop_duplicates(subset='spotify_id').copy()
print('Tamaño sin duplicados de canciones: ', data_ana_corona.shape)

# Para analizar teniendo en cuenta los streams totales y que esté ponderado, calculamos los pesos de cada canción respecto a los streams totales
# Importante hacerlo una vez quitados los duplicados
# Versión optimizada corta y sin warnings

data_ana_corona_2020 = data_ana_corona.loc[data_ana_corona['year'] == 2020]
data_ana_corona_2019 = data_ana_corona.loc[data_ana_corona['year'] == 2019]
data_ana_corona_2018 = data_ana_corona.loc[data_ana_corona['year'] == 2018]
data_ana_corona_2017 = data_ana_corona.loc[data_ana_corona['year'] == 2017]

data_ana_corona.loc[data_ana_corona.year == 2020,
                    'streamstotal_weights'] = data_ana_corona_2020['Streamstotal']/data_ana_corona_2020['Streamstotal'].sum()
data_ana_corona.loc[data_ana_corona.year == 2019,
                    'streamstotal_weights'] = data_ana_corona_2019['Streamstotal']/data_ana_corona_2019['Streamstotal'].sum()
data_ana_corona.loc[data_ana_corona.year == 2018,
                    'streamstotal_weights'] = data_ana_corona_2018['Streamstotal']/data_ana_corona_2018['Streamstotal'].sum()
data_ana_corona.loc[data_ana_corona.year == 2017, 
                    'streamstotal_weights'] = data_ana_corona_2017['Streamstotal']/data_ana_corona_2017['Streamstotal'].sum()

# Rehago la función para sacar las medias ponderadas
def mediapon_features (feature, dataframe_period):
    media_compar = dataframe_period.loc[:, ['year', feature, 'streamstotal_weights']]
    media_compar["mediaSpecial"] = media_compar[feature] * media_compar['streamstotal_weights']
    media_compar = media_compar.groupby(['year']).sum()
    media_compar.reset_index(inplace=True)
    media_compar = media_compar.drop(columns=[feature, 'streamstotal_weights', 'year'])
    media_compar["features"] = feature  
    return media_compar

media_lenght = mediapon_features('length', data_ana_corona)[3:]
media_popularity = mediapon_features('popularity', data_ana_corona)[3:]
media_acousticness = mediapon_features('acousticness', data_ana_corona)[3:]
media_danceability = mediapon_features('danceability', data_ana_corona)[3:]
media_energy = mediapon_features('energy', data_ana_corona)[3:]
media_instrumentalness = mediapon_features('instrumentalness', data_ana_corona)[3:]
media_liveness = mediapon_features('liveness', data_ana_corona)[3:]
media_speechiness = mediapon_features('speechiness', data_ana_corona)[3:]
media_valence = mediapon_features('valence', data_ana_corona)[3:]
media_loudness = mediapon_features('loudness', data_ana_corona)[3:]
media_tempo = mediapon_features('tempo', data_ana_corona)[3:]
data_ana_special = pd.concat([
           media_lenght,
           media_popularity,
           media_acousticness,
           media_danceability,
           media_energy,
           media_instrumentalness,
           media_liveness,
           media_speechiness,
           media_valence,
           media_loudness,
           media_tempo,
          ], axis=0)
data_ana_special

Tamaño quitando columnas que sobran:  (48066, 36)
Tamaño sin duplicados de canciones:  (1581, 36)


/Users/victormac/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,mediaSpecial,features
3,0.335215,length
3,0.772030,popularity
3,0.218602,acousticness
3,0.745346,danceability
3,0.703478,energy
3,0.007287,instrumentalness
3,0.149582,liveness
3,0.136460,speechiness
3,0.638213,valence
3,0.855453,loudness


## Tendencia anual de features

También quiero aprovechar y sacar la tendencia anual con comparativas de las medias en periodo pandemia en 2020 y mismo periodo en años anteriores

In [482]:
# Rehago la función anterior para sacar las medias ponderadas y quedarme con los datos que me interesan
def mediapon_features_tenden (feature, dataframe_period):
    media_compar = dataframe_period.loc[:, ['year', feature, 'streamstotal_weights']]
    media_compar["mediaSpecial"] = media_compar[feature] * media_compar['streamstotal_weights']
    media_compar = media_compar.groupby(['year']).sum()
    media_compar.reset_index(inplace=True)
    media_compar = media_compar.drop(columns=[feature, 'streamstotal_weights'])
    media_compar["features"] = feature  
    return media_compar

media_lenght_tenden = mediapon_features_tenden('length', data_ana_corona)
media_popularity_tenden = mediapon_features_tenden('popularity', data_ana_corona)
media_acousticness_tenden = mediapon_features_tenden('acousticness', data_ana_corona)
media_danceability_tenden = mediapon_features_tenden('danceability', data_ana_corona)
media_energy_tenden = mediapon_features_tenden('energy', data_ana_corona)
media_instrumentalness_tenden = mediapon_features_tenden('instrumentalness', data_ana_corona)
media_liveness_tenden = mediapon_features_tenden('liveness', data_ana_corona)
media_speechiness_tenden = mediapon_features_tenden('speechiness', data_ana_corona)
media_valence_tenden = mediapon_features_tenden('valence', data_ana_corona)
media_loudness_tenden = mediapon_features_tenden('loudness', data_ana_corona)
media_tempo_tenden = mediapon_features_tenden('tempo', data_ana_corona)
                                               
data_ana_special_tenden = pd.concat([media_lenght_tenden,
           media_popularity_tenden,
           media_acousticness_tenden,
           media_danceability_tenden,
           media_energy_tenden,
           media_instrumentalness_tenden,
           media_liveness_tenden,
           media_speechiness_tenden,
           media_valence_tenden,
           media_loudness_tenden,
           media_tempo_tenden,
          ], axis=0)

#Exportamos para analizar en Tableau
data_ana_special_tenden.to_csv("tableau_graph_and_analysis/data_ana_playlist_tendence_totableau.csv", sep = ',')
data_ana_special_tenden.head(10)

,year,mediaSpecial,features
0,2017,0.376435,length
1,2018,0.356821,length
2,2019,0.373368,length
3,2020,0.335215,length
0,2017,0.375080,popularity
1,2018,0.369754,popularity
2,2019,0.590811,popularity
3,2020,0.772030,popularity
0,2017,0.174298,acousticness
1,2018,0.199013,acousticness


## Recarga de datos 

De las extracciones anteriores que han sido largas y hemos exportado en CSV, las volvemos a cargar

In [484]:
# Cargamos dataframes guardados en extracción anterior
data_ana_playlist_songs100 = pd.read_csv("data_ana_playlist_songs100.csv", sep = ',')
data_ana_playlist_songs100 = data_ana_playlist_songs100.drop(columns=['Unnamed: 0', 'time_signature', 'release_date', 'release_date_year'])

data_ana_playlist_songs100_2019 = pd.read_csv("data_ana_playlist_songs100_2019.csv", sep = ',')
data_ana_playlist_songs100_2019 = data_ana_playlist_songs100_2019.drop(columns=['Unnamed: 0', 'time_signature', 'release_date','release_date_year'])

data_ana_playlist_songs10000 = pd.read_csv("data_ana_playlist_songs10000.csv", sep = ',')
data_ana_playlist_songs10000 = data_ana_playlist_songs10000.drop(columns=['Unnamed: 0', 'time_signature', 'release_date','release_date_year'])


Calculamos las medias y las exportamos para hacer el análisis visual desde Tableau

In [431]:
#Calculamos medias de 
data_ana_playlist_means_songs100 = data_ana_playlist_songs100.mean().reset_index().rename(columns={"index": "features", 0: "mediaTop100_pandemia"})
data_ana_playlist_means_songs100_2019 = data_ana_playlist_songs100_2019.mean().reset_index().rename(columns={"index": "features", 0: "mediaTop100_2019"})
data_ana_playlist_means_songs10000 = data_ana_playlist_songs10000.mean().reset_index().rename(columns={"index": "features", 0: "mediaTop10000_pandemia"})

#Hacemos merge de los 4 dataframes
from functools import reduce
dataframes_merge = [data_ana_playlist_means_songs100,
                    data_ana_playlist_means_songs100_2019,
                    data_ana_playlist_means_songs10000,
                    data_ana_special]
data_ana_playlist_means_all = reduce(lambda left,right: pd.merge(left,right,on='features'), dataframes_merge)


#Exportamos para analizar en Tableau
data_ana_playlist_means_all.to_csv("tableau_graph_and_analysis/data_ana_playlist_songs_totableau.csv", sep = ',')

El tableau está en la carpeta raíz (spotify-musicanalysis-coronavirus/tableau_graph_and_analysis/Tableau_Analysis4_playlists.twb) pero he incluido más abajo una imagen del mismo.
En el gráfico vemos comparadas cuatro medias:
- **A**, Top100 de las playlists de la Pandemia en 2020
- **B**, Top10000 de las playlists de la Pandemia en 2020. Este cálculo es lógico que al abarcar mucho más música, abarca más variedad, entonces, siempre tenderá a suavizar la media anterior, nos ayuda a evitar los lanzamientos puntuales que pueden acaparar el top100
- **C**, Las medias de nuestro propio análisis, que están ponderadas con los Streams totales, y teniendo en cuenta que las canciones están sacadas del top200 de España) en 2020
- **Z**, Top100 2019, de la playlist sacado de Spotify que recoge las 100 canciones más escuchadas de dicho año

## Análisis del dashboard

Consideramos que la diferencia entre el 2019 y 2020 es clara cuando las tres columnas de diferencias son iguales en su valor es negativo o positivo. **Ojo e importante!** Hay que recordar, y es algo que remarcamos en el análisis inicial con features, que si, por ejemplo en la feature danceability ya existía una tendencia de que ha ido aumentando año tras año la media de dicha feature, es decir, que año tras año cada vez se escucha más cantidad de música bailable, el hecho de que en este análisis comparativo con las playlists haya habido un aumento de la media respecto al 2019 en las playlists de la pandemia, puede no tener que ver con que haya habido un cambio real por las circunstancias de la pandemia, si no por la tendencia que ya se venía viendo desde años atrás. Por eso es importante tener en cuenta las tendencias y a veces es díficil determinar hasta qué punto la pandemia ha influido o no en la tendencia o es una continuidad de la misma. Por otro lado, es importante decir que los datos de tendencias son con datos de España, lo suyo sería saber la tendencia a nivel mundial, pero de momento esto nos puede dar una idea.

Analizamos las features de las canciones una a una:

- **Acousticness:** Parece que respecto al 2019, se escuchó menos música acústica durante la pandemia y teniendo en cuenta que la tendencia por año marca que cada vez se escucha más música más acústica hasta el 2020, podemos decir que sí, que la pandemia ha afectado provocando que se escuche música menos acústica
- **Danceability:** Las medias de nuestro análisis propio y del top100, recogen que se escuchó más música más bailable, sin embargo en el top10000 no es así. Si a esto le sumamos que en la tendencia se nos muestra que hay bajada del valor para concretamente el año 2020 y en el periodo Pandemia (y varía respecto al Periodo normal), como deducimos en el Analisis inicial de features, podemos decir que se ha escuchado menos música más bailable.
- **Energy:** La comparación por años nos dice que se escuchaba cada vez menos música más enérgica hasta el año 2020 y el ana´lsiis de playlist nos confirma que sí, durante la pandemia se ha escuchado música más enérgica
- **Instrumentalness:** Sin datos relevantes, tampoco es una feature que aporte mucho
- **Length:** No day datos claros para sacar en el periodo pandemia.
- **Liveness:** Hay un aumento de escucha de que tiene una probabilidad de ser en directo. Aunque la tendencia es que ha ido bajando en los últimos años.
- **Popularity:** Es curioso que respecto al 2019, en el periodo pandemia se escuche menos música más popular, sin embargo, la tendencia por años nos dice que se escucha más música más popular. ¿Hemos dado un giro en la pandemia y preferimos lo mismo popular?
- **Loudness:** Parece que se ha escuchado música más ruidosa y con más volumen durante la pandemia aunque la tendencia es simialr en años.
- **Speechiness:** Aunque la tendencia en años es que se escuche más música con más texto hablado (ejemplo rap), en la pandemia parece que hemso evitado este tipo de música.
- **Tempo:** Sin datos relevantes, tampoco es una feature que aporte mucho
- **Valence:** La tendencia entre años es que crece y comparaticamente en la pandemia se ve que sí, hemos escuchado más música más positiva

Después del gráfico dejo unas conclusiones.

<img src="tableau_graph_and_analysis/dashboard_playlist.png" style="width: 1200px;">

## Conclusiones:

Si juntamos este análisis con el inicial que hicimos de Features, podemos acercarnos más a la realidad. Aunque seguimos sin poder afirmar claramente ciertos cambios en el tipo de música que escuchamos, sí podemos decir que durante la pandemia hemos escuchado:
- Menos música más acústica (cambiando la tendencia anterior)
- Menos música más bailable, con menor tempo, ritmo y fuerza de los "beats" (pero la tendencia ya iba en aumento en años anteriores, cuidado.)
- Más música más enérgica, con más velocidad, sonoridad y ruido (cambiando la tendencia anterior)
- Más música más positiva, de felicidad, eufórica (siguiendo un poco la tendencia)

**Si pensamos en esto podemos deducir que en un periodo de confinamiento, donde las personas están encerradas en casa, en una situación de incertidumbre, tal vez mediante la música, buscamos formas de sacar la energía contenida y animarnos con música positiva. Eso sí, ¿puede que escuchemos menos música más bailable porque podemos bailar menos?**

Otras conclusiones curiosidas:
- No está claro que durante la pandemia se haya escuchado más música en directo a pesar de estar encerrados en casa.
- Aunque cada vez escuchamos más música más popular y de éxitos, durante la pandemia no fue tanto así